In [ ]:
### Helper Functions

import re

def check_patterns(response):
    # Define all required patterns
    patterns = {
        'answer': r'<answer>(.*?)</answer>',
        'reflection': r'<reflection>(.*?)</reflection>',
        'steps': r'<step>(.*?)</step>',
        'count': r'<count>(.*?)</count>'
    }

    # Check all patterns exist
    matches = {
        'answer': re.search(patterns['answer'], response, re.DOTALL),
        'reflection': re.search(patterns['reflection'], response, re.DOTALL),
        'steps': re.findall(patterns['steps'], response, re.DOTALL),
        'count': re.findall(patterns['count'], response, re.DOTALL)
    }

    # Return None if any pattern is missing
    if not all([matches['answer'], matches['reflection'], matches['steps'], matches['count']]):
        return "Missing"

    # Extract and return the matched content
    return response



def parse_response(response):
    """Parses the response from the model."""

    # Checking if the response includes all the required patterns
    response_check = check_patterns(response)
    if response_check == "Missing":
        # Extract clarification content
        clarification_match = re.search(r'<clarification>(.*?)</clarification>', response, re.DOTALL)
        clarification = clarification_match.group(1).strip() if clarification_match else ""
        if not clarification:
            clarification=response
        return "", "", [], clarification
    else:
        # Extract answer, reflection, and steps
        answer_match = re.search(r'<answer>(.*?)</answer>', response, re.DOTALL)
        reflection_match = re.search(r'<reflection>(.*?)</reflection>', response, re.DOTALL)

        answer = answer_match.group(1).strip() if answer_match else ""
        reflection = reflection_match.group(1).strip() if reflection_match else ""
        steps = re.findall(r'<step>(.*?)</step>', response, re.DOTALL)

        return answer, reflection, steps, ""



def handle_clarification(client, original_question, clarification):
    if clarification:
        print("Clarification needed:")
        print(clarification)

        while True:

                additional_info = input("Please provide additional clarification: ")

                # Create a new completion with the original question and additional clarification
                new_completion = client.chat.completions.create(
                    model="/LLM_32T/pretrained_weights/preview_model/VietCoMath_8B_instruct_2024_11",
                    messages=[
                        {"role": "system", "content": DEFAULT_SYSTEM_PROMPT},
                        {"role": "user", "content": original_question},
                        {"role": "assistant", "content": clarification},
                        {"role": "user", "content": f"Additional clarification: {additional_info}"}
                    ],
                    temperature=0.6,
                    top_p=0.9,
                    max_tokens=4096,
                )

                # Get the new generated text
                new_generated_text = new_completion.choices[0].message.content

                # Parse the new response
                new_answer, new_reflection, new_steps, new_clarification = parse_response(new_generated_text)

                # Print the new results
                print("\n------------New Response-------------")
                print("New Clarification:", new_clarification)
                print("------------Internal Thinking-------------")
                print(new_steps)
                print(new_reflection)
                print("------------End of Internal Thinking-------------\n")
                print("------------New Answer-------------")
                print(new_answer)
                print("------------End of Answer-------------")

                # Recursively handle any new clarification
                handle_clarification(client, original_question, new_clarification)
                break





In [ ]:
from openai import OpenAI


DEFAULT_SYSTEM_PROMPT =""


### Testing Question
'''
Testing Mộ số 50 câu đố toán học vui, câu đố mẹo toán học giúp rèn luyện tư duy
https://voh.com.vn/song-dep/cau-do-toan-hoc-443351.html
'''


client = OpenAI(
    base_url="http://13.65.249.11:8887/v1",
    api_key="token-abc123",

)
completion = client.chat.completions.create(
  model="/LLM_32T/pretrained_weights/preview_model/VietCoMath_8B_instruct_2024_11",
  messages=[
    {"role": "system", "content": f"{DEFAULT_SYSTEM_PROMPT}"},
    {"role": "user", "content": "Có 100 sinh viên đỗ đại học. Trong số đó, có 55 sinh viên chọn âm nhạc, 44 sinh viên chọn thể thao, và 20 sinh viên chọn cả 2. Hỏi có bao nhiêu sinh viên không chọn âm nhạc, cũng không chọn thể thao.?"}
  ],
   temperature=0.6,
    top_p=0.9,
    max_tokens=4096,
)

'''
Đáp Án:
Chúng ta sẽ xử lý câu đố này như sau. Có 20 sinh viên chọn cả âm nhạc và thể thao, vì thế số sinh viên còn lại chọn âm nhạc là 35, và thể thao là 24.
Vậy, số sinh viên không chọn cả 2 là: 100 – 20 (số học sinh chọn cả âm nhạc và thể thao) – 35 (âm nhạc) – 24 (thể thao) = 21 sinh viên.
'''

generated_text=completion.choices[0].message.content
answer, reflection, steps, clarification = parse_response(generated_text)

print(clarification)
print("------------Internal Thinking-------------")
print(steps)
print(reflection)
print("------------End of Internal Thinking-------------\n")

print("------------Final Answer-------------")
print(answer)
print("------------End of Answer-------------")

# IF Clarification Requirement Remember to adding back the Original Question
original_question="Có 100 sinh viên đỗ đại học. Trong số đó, có 55 sinh viên chọn âm nhạc, 44 sinh viên chọn thể thao, và 20 sinh viên chọn cả 2. Hỏi có bao nhiêu sinh viên không chọn âm nhạc, cũng không chọn thể thao.?"

handle_clarification(client, original_question, clarification)

Để giải quyết câu hỏi này, chúng ta cần xác định số sinh viên không chọn âm nhạc và không chọn thể thao. Bạn có thể xác nhận rằng thông tin đã cung cấp là đầy đủ và chính xác không?
------------Internal Thinking-------------
[]

------------End of Internal Thinking-------------

------------Final Answer-------------

------------End of Answer-------------
Clarification needed:
Để giải quyết câu hỏi này, chúng ta cần xác định số sinh viên không chọn âm nhạc và không chọn thể thao. Bạn có thể xác nhận rằng thông tin đã cung cấp là đầy đủ và chính xác không?
Please provide additional clarification: đó là thông tin đầy đủ!

------------New Response-------------
New Clarification: 
------------Internal Thinking-------------
[' Bước 1: Xác định các tập hợp và số lượng sinh viên trong mỗi tập hợp.\n- Tập H1: Sinh viên chọn âm nhạc (55 sinh viên)\n- Tập H2: Sinh viên chọn thể thao (44 sinh viên)\n- Tập H1 ∩ H2: Sinh viên chọn cả âm nhạc và thể thao (20 sinh viên)\n', ' Bước 2: Sử dụng công thứ